Imports

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Read the file

In [19]:
df = pd.read_csv('https://raw.githubusercontent.com/HoverHander/CS4661-Final-Project/main/Datasets/train.csv')

Replace categorical features with dummy columns

In [20]:
fill_columns = ['LotFrontage', 'GarageYrBlt', 'MasVnrArea']
df[fill_columns] = df[fill_columns].fillna(0)

# should identify categorical features
categorical_features = df.select_dtypes(include=['object']).columns
df_encoded = pd.get_dummies(df, columns=categorical_features)
df_encoded[0::10]

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
10,11,20,70.0,11200,5,5,1965,1965,0.0,906,...,0,0,0,1,0,0,0,0,1,0
20,21,60,101.0,14215,8,5,2005,2006,380.0,0,...,0,1,0,0,0,0,0,0,0,1
30,31,70,50.0,8500,4,4,1920,1950,0.0,0,...,0,0,0,1,0,0,0,0,1,0
40,41,20,84.0,8658,6,5,1965,1965,101.0,643,...,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410,1411,60,79.0,12420,7,5,2001,2001,0.0,666,...,0,0,0,1,0,0,0,0,1,0
1420,1421,60,90.0,11700,6,6,1968,1968,420.0,404,...,0,0,0,1,0,0,0,0,1,0
1430,1431,60,60.0,21930,5,5,2005,2005,0.0,0,...,0,0,0,1,0,0,0,0,1,0
1440,1441,70,79.0,11526,6,7,1922,1994,0.0,0,...,0,0,0,1,0,0,0,0,1,0


Set up feature matrix and label vector

In [21]:
X = df_encoded.drop('SalePrice', axis=1)
y = df_encoded['SalePrice']
print('X shape: ', X.shape)
print('y shape: ', y.shape)

X shape:  (1460, 289)
y shape:  (1460,)


Split the dataset

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# print the size of the traning set:
print(X_train.shape)
print(y_train.shape)

# print the size of the testing set:
print(X_test.shape)
print(y_test.shape)

(1168, 289)
(1168,)
(292, 289)
(292,)


Random Forest Regression

In [23]:
from sklearn.ensemble import RandomForestRegressor

house_rfr = RandomForestRegressor()


Training and predicting

In [24]:
house_rfr.fit(X_train, y_train)
y_prediction = house_rfr.predict(X_test)
print(y_prediction)

[268438.77 178533.9  287961.88 219158.53 209321.06 216962.7  214805.87
 201479.   155718.6  355421.67 204290.63 185400.3   96002.33 136192.
 108856.61 235659.57 267471.4  282411.79 189936.   296940.37 336596.38
 171090.83 174547.65 238135.   370176.94 225544.02 149393.   269558.69
 216794.43 407383.61 324773.41 167827.   269270.26 131725.   185474.55
 221866.83 124369.58 304752.15 324717.67 360903.46 161381.82 187215.
 148450.    99299.5  110650.88 230281.31  96469.   169757.34  95919.5
 145510.25 150424.5  140424.5  236372.33 174539.19 129161.1  292450.4
 128034.04 270266.43 387317.67 126410.5  174267.55 138296.   297602.61
 146505.25 362210.36 152010.   222540.78 120640.   193566.1  128875.65
 217706.58 109431.3  110772.5  140211.8  148582.14 198762.25 169607.75
 275648.28 162576.   233939.28 300258.97 182998.54 194003.96 214823.75
 150519.52 191424.89 172689.   229751.5  211544.9  154888.32 160335.89
 223440.56 201221.94 181258.62 177610.52 181764.9  107235.5  157762.
 140100.45 185

RMSE

In [25]:
from sklearn import metrics

mse = metrics.mean_squared_error(y_test, y_prediction)
rmse = np.sqrt(mse)

print(rmse)

27461.851197782213


Trying with cross validation

In [26]:
mse_list = cross_val_score(house_rfr, X, y, cv=10, scoring='neg_mean_squared_error')
print(mse_list)

[-6.41444404e+08 -7.22611293e+08 -4.79345284e+08 -1.51460298e+09
 -1.14627796e+09 -6.63840353e+08 -6.09132394e+08 -5.36560575e+08
 -1.75158239e+09 -7.52246940e+08]


Make the results positive, calculate the square root and mean

In [27]:
mse_list_pos = -mse_list
rmse_list = np.sqrt(mse_list_pos)
print(rmse_list.mean())

28976.528105341586
